In [1]:
# --------------------  IMPORT MODULES AND PACKAGES  ------------------------
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_percentage_error
import sqlite3 # for sqlite connectivity
 


In [2]:
# ------------------- Connecting sql server ----------------------

conn = sqlite3.connect('your_database.db')
curser = conn.cursor()

In [3]:
# --------------------  Creating an dataframe of Sales data -----------------------
sales = "SELECT * FROM sales"
df = pd.read_sql_query(sales,conn)
print(df)

                   sales_id      s_date  quantity  p_price  p_discount  \
0        202007011331747686  2020-07-01     0.565     14.0           0   
1        202007011331748913  2020-07-01     0.243     14.0           0   
2        202007011331749910  2020-07-01     0.551     14.0           0   
3        202007011519841799  2020-07-01     0.085     25.8           0   
4        202007011519846043  2020-07-01     0.089     25.8           0   
...                     ...         ...       ...      ...         ...   
515639  2023063099414972798  2023-06-30     1.225      6.0           0   
515640  2023063099414985246  2023-06-30     0.269      6.0           0   
515641  2023063099414991702  2023-06-30     0.571      6.0           0   
515642  2023063099414994356  2023-06-30     0.470      6.0           0   
515643  2023063099414999837  2023-06-30     0.621      6.0           0   

               p_name    p_id  
0             Spinach  133174  
1             Spinach  133174  
2             S

In [4]:
# -------------------- slicing the df for only needed features -------------------------------

df = df.iloc[:,1:]
df

,s_date,quantity,p_price,p_discount,p_name,p_id
0,2020-07-01,0.565,14.0,0,Spinach,133174
1,2020-07-01,0.243,14.0,0,Spinach,133174
2,2020-07-01,0.551,14.0,0,Spinach,133174
3,2020-07-01,0.085,25.8,0,Millet Pepper,151984
4,2020-07-01,0.089,25.8,0,Millet Pepper,151984
...,...,...,...,...,...,...
515639,2023-06-30,1.225,6.0,0,Eggplant (2),994149
515640,2023-06-30,0.269,6.0,0,Eggplant (2),994149
515641,2023-06-30,0.571,6.0,0,Eggplant (2),994149
515642,2023-06-30,0.470,6.0,0,Eggplant (2),994149


In [5]:
# --------------------------- from sales data generating an daily sales quantity of each product -----------------------

# Group by date and item name, then sum the sales
daily_sales = df.groupby(['s_date', 'p_id'])['quantity'].sum().reset_index()

# Print or use the daily_sales Series
daily_sales

,s_date,p_id,quantity
0,2020-07-01,133174,5.472
1,2020-07-01,151984,4.383
2,2020-07-01,177880,22.402
3,2020-07-01,342213,32.579
4,2020-07-01,635317,34.554
...,...,...,...
14394,2023-06-30,592508,8.470
14395,2023-06-30,635317,16.900
14396,2023-06-30,760349,7.036
14397,2023-06-30,827839,12.884


In [6]:
# -----------------------  sales count for each day sales of each product ---------------------------------

salesCount = df.groupby(['s_date','p_id'])['p_id'].value_counts().rename('Sales_count').reset_index()
salesCount

,s_date,p_id,Sales_count
0,2020-07-01,133174,14
1,2020-07-01,151984,45
2,2020-07-01,177880,58
3,2020-07-01,342213,71
4,2020-07-01,635317,79
...,...,...,...
14394,2023-06-30,592508,21
14395,2023-06-30,635317,37
14396,2023-06-30,760349,15
14397,2023-06-30,827839,32


In [7]:
#------------------------ merging the sales count and daily sales for more understantability ----------------------------

daily_sales = pd.merge(daily_sales,salesCount,on = ['s_date' , 'p_id'],how='left')
daily_sales


,s_date,p_id,quantity,Sales_count
0,2020-07-01,133174,5.472,14
1,2020-07-01,151984,4.383,45
2,2020-07-01,177880,22.402,58
3,2020-07-01,342213,32.579,71
4,2020-07-01,635317,34.554,79
...,...,...,...,...
14394,2023-06-30,592508,8.470,21
14395,2023-06-30,635317,16.900,37
14396,2023-06-30,760349,7.036,15
14397,2023-06-30,827839,12.884,32


In [8]:
# ----------------- finding is the product has discount at that date --------------

dis = df.groupby(['s_date' , 'p_id'])['p_discount'].sum().reset_index()
dis

,s_date,p_id,p_discount
0,2020-07-01,133174,0
1,2020-07-01,151984,0
2,2020-07-01,177880,31
3,2020-07-01,342213,0
4,2020-07-01,635317,0
...,...,...,...
14394,2023-06-30,592508,0
14395,2023-06-30,635317,0
14396,2023-06-30,760349,0
14397,2023-06-30,827839,0


In [9]:
# ------------------- if it has set the value of discount to q else 0 --------------------

dis['p_discount'] = dis['p_discount'].apply(lambda x: 1 if x > 0 else 0)
dis

,s_date,p_id,p_discount
0,2020-07-01,133174,0
1,2020-07-01,151984,0
2,2020-07-01,177880,1
3,2020-07-01,342213,0
4,2020-07-01,635317,0
...,...,...,...
14394,2023-06-30,592508,0
14395,2023-06-30,635317,0
14396,2023-06-30,760349,0
14397,2023-06-30,827839,0


In [10]:
#---------------------- Merge 'dis' with 'df' on 'Date' and 'Item Name'----------------------------

daily_sales = pd.merge(daily_sales, dis, on=['s_date', 'p_id'], how='left')

# Display the merged DataFrame
daily_sales

,s_date,p_id,quantity,Sales_count,p_discount
0,2020-07-01,133174,5.472,14,0
1,2020-07-01,151984,4.383,45,0
2,2020-07-01,177880,22.402,58,1
3,2020-07-01,342213,32.579,71,0
4,2020-07-01,635317,34.554,79,0
...,...,...,...,...,...
14394,2023-06-30,592508,8.470,21,0
14395,2023-06-30,635317,16.900,37,0
14396,2023-06-30,760349,7.036,15,0
14397,2023-06-30,827839,12.884,32,0


In [11]:
# -------------------- finding the price of each product at that day -----------------------

price = df.groupby(['s_date','p_id','p_price'])['p_id'].value_counts().reset_index()
price

,s_date,p_id,p_price,count
0,2020-07-01,133174,14.0,14
1,2020-07-01,151984,25.8,45
2,2020-07-01,177880,4.0,31
3,2020-07-01,177880,6.0,27
4,2020-07-01,342213,6.0,71
...,...,...,...,...
16933,2023-06-30,592508,12.0,21
16934,2023-06-30,635317,11.2,37
16935,2023-06-30,760349,8.0,15
16936,2023-06-30,827839,3.4,32


In [12]:
# ------------------------------- Drop duplicate rows based on 'Date' and 'Item Name', keeping the first occurrence ------------------------
#  bcz it has many parices on single product at the same day ....  

price = price.drop_duplicates(subset=['s_date', 'p_id'], keep='first')
price

,s_date,p_id,p_price,count
0,2020-07-01,133174,14.0,14
1,2020-07-01,151984,25.8,45
2,2020-07-01,177880,4.0,31
4,2020-07-01,342213,6.0,71
5,2020-07-01,635317,14.0,79
...,...,...,...,...
16933,2023-06-30,592508,12.0,21
16934,2023-06-30,635317,11.2,37
16935,2023-06-30,760349,8.0,15
16936,2023-06-30,827839,3.4,32


In [13]:
# ----------------- merging the price with the daily sales ------------------------

daily_sales = pd.merge(daily_sales,price,on=['s_date','p_id'],how='left')
daily_sales

,s_date,p_id,quantity,Sales_count,p_discount,p_price,count
0,2020-07-01,133174,5.472,14,0,14.0,14
1,2020-07-01,151984,4.383,45,0,25.8,45
2,2020-07-01,177880,22.402,58,1,4.0,31
3,2020-07-01,342213,32.579,71,0,6.0,71
4,2020-07-01,635317,34.554,79,0,14.0,79
...,...,...,...,...,...,...,...
14394,2023-06-30,592508,8.470,21,0,12.0,21
14395,2023-06-30,635317,16.900,37,0,11.2,37
14396,2023-06-30,760349,7.036,15,0,8.0,15
14397,2023-06-30,827839,12.884,32,0,3.4,32


In [14]:
daily_sales = daily_sales.drop(columns=['Sales_count','count'],axis=1)

In [15]:
daily_sales

,s_date,p_id,quantity,p_discount,p_price
0,2020-07-01,133174,5.472,0,14.0
1,2020-07-01,151984,4.383,0,25.8
2,2020-07-01,177880,22.402,1,4.0
3,2020-07-01,342213,32.579,0,6.0
4,2020-07-01,635317,34.554,0,14.0
...,...,...,...,...,...
14394,2023-06-30,592508,8.470,0,12.0
14395,2023-06-30,635317,16.900,0,11.2
14396,2023-06-30,760349,7.036,0,8.0
14397,2023-06-30,827839,12.884,0,3.4


In [16]:
# --------------------------  extracting featres from the date  -------------------------------------

# Assuming 'daily_sales' DataFrame exists and has a 'Date' column
daily_sales['s_date'] = pd.to_datetime(daily_sales['s_date'])
daily_sales['monthOfyear'] = daily_sales['s_date'].dt.month
daily_sales['DayOfWeek'] = daily_sales['s_date'].dt.dayofweek  # Monday=0, Sunday=6
daily_sales['DayOfMonth'] = daily_sales['s_date'].dt.day
daily_sales['year'] = daily_sales['s_date'].dt.year
daily_sales['Weekend'] = daily_sales['DayOfWeek'].apply(lambda x: 1 if x in [5, 6] else 0)
# Calculate the week of the month
daily_sales['WeekOfMonth'] = (daily_sales['DayOfMonth'] - 1) // 7 + 1
daily_sales['quater'] = daily_sales['s_date'].dt.quarter
daily_sales

,s_date,p_id,quantity,p_discount,p_price,monthOfyear,DayOfWeek,DayOfMonth,year,Weekend,WeekOfMonth,quater
0,2020-07-01,133174,5.472,0,14.0,7,2,1,2020,0,1,3
1,2020-07-01,151984,4.383,0,25.8,7,2,1,2020,0,1,3
2,2020-07-01,177880,22.402,1,4.0,7,2,1,2020,0,1,3
3,2020-07-01,342213,32.579,0,6.0,7,2,1,2020,0,1,3
4,2020-07-01,635317,34.554,0,14.0,7,2,1,2020,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...
14394,2023-06-30,592508,8.470,0,12.0,6,4,30,2023,0,5,2
14395,2023-06-30,635317,16.900,0,11.2,6,4,30,2023,0,5,2
14396,2023-06-30,760349,7.036,0,8.0,6,4,30,2023,0,5,2
14397,2023-06-30,827839,12.884,0,3.4,6,4,30,2023,0,5,2


In [17]:
# # ------------------------- encording the name -------------------------

# # Create an instance of LabelEncoder
# encoder = LabelEncoder()

# # Fit and transform the 'p_name' column
# daily_sales['p_id'] = encoder.fit_transform(daily_sales['p_id'])

# daily_sales

In [18]:
# -----------------------------  spliting x and y axis --------------------------

x = daily_sales.drop(columns = ['s_date','quantity'])
y = daily_sales.iloc[:,2]

In [19]:
x

,p_id,p_discount,p_price,monthOfyear,DayOfWeek,DayOfMonth,year,Weekend,WeekOfMonth,quater
0,133174,0,14.0,7,2,1,2020,0,1,3
1,151984,0,25.8,7,2,1,2020,0,1,3
2,177880,1,4.0,7,2,1,2020,0,1,3
3,342213,0,6.0,7,2,1,2020,0,1,3
4,635317,0,14.0,7,2,1,2020,0,1,3
...,...,...,...,...,...,...,...,...,...,...
14394,592508,0,12.0,6,4,30,2023,0,5,2
14395,635317,0,11.2,6,4,30,2023,0,5,2
14396,760349,0,8.0,6,4,30,2023,0,5,2
14397,827839,0,3.4,6,4,30,2023,0,5,2


In [20]:
y

0         5.472
1         4.383
2        22.402
3        32.579
4        34.554
          ...  
14394     8.470
14395    16.900
14396     7.036
14397    12.884
14398    14.365
Name: quantity, Length: 14399, dtype: float64

In [21]:
# ------------------------- finding the best model -----------------------

# Import necessary libraries (assuming they are already installed)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor #, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_percentage_error

# ... (Your existing code for data preprocessing) ...

# Split data into training and testing sets (assuming x and y are defined)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# x_train = sc.fit_transform(x_train)
# X_test = sc.transform(x_test)

# Initialize regression models
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Lasso Regression": Lasso(),
    "Elastic Net": ElasticNet(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(),
    "Support Vector Regression": SVR(),
    "K-Nearest Neighbors": KNeighborsRegressor()
}

results = {}

for name, model in models.items():
    model.fit(x_train, y_train)  # Train the model
    y_pred = model.predict(x_test) # Make predictions

    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mase = mean_absolute_percentage_error(y_test,y_pred)
    results[name] = {"MSE": mse, "R-squared": r2,"mase" : mase}

# Find the best model based on MSE (lower is better)
best_model_mse = min(results, key=lambda k: results[k]["MSE"])
best_model_r2 = max(results, key=lambda k: results[k]["R-squared"])
best_model_mase = min(results, key=lambda k: results[k]["mase"])


print("Model Performance:")
for name, metrics in results.items():
    print(f"{name}: MSE={metrics['MSE']:.4f}, R-squared={metrics['R-squared']:.4f}, mase={metrics['mase']:.4f}")

print(f"\nBest model based on lowest MSE: {best_model_mse}")
print(f"\nBest model based on highest R-squared: {best_model_r2}")
print(f"\nBest model based on highest best_model_mase: {best_model_mase}")

Model Performance:
Linear Regression: MSE=281.0948, R-squared=0.1185, mase=2.5781
Ridge Regression: MSE=281.0965, R-squared=0.1185, mase=2.5782
Lasso Regression: MSE=296.2690, R-squared=0.0709, mase=2.7434
Elastic Net: MSE=294.5908, R-squared=0.0762, mase=2.7263
Decision Tree: MSE=182.2122, R-squared=0.4286, mase=1.0900
Random Forest: MSE=90.9277, R-squared=0.7149, mase=1.0398
Support Vector Regression: MSE=338.7789, R-squared=-0.0624, mase=1.9107
K-Nearest Neighbors: MSE=129.2022, R-squared=0.5948, mase=1.2535

Best model based on lowest MSE: Random Forest

Best model based on highest R-squared: Random Forest

Best model based on highest best_model_mase: Random Forest


In [22]:
# ---------------------------  randome forest is the best model for this project ,----------------------------

# model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
# Initialize and train the RandomForestRegressor
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)
rf_model = RandomForestRegressor()
rf_model.fit(x, y)   # using the entire dataset to train the model 

RandomForestRegressor()

In [23]:
#  --------------------  importing product table --------------------------------


product = "SELECT * FROM product"
product = pd.read_sql_query(product,conn)
product

,p_id,p_name,p_price,p_discount
0,133174,Spinach,10.28,0
1,151984,Millet Pepper,27.18,0
2,151990,Needle Mushroom (Box),3.06,0
3,177880,Hongshujian,7.33,0
4,184283,Yunnan Lettuce (Bag),3.41,1
5,192609,Naibaicai,4.73,0
6,247321,Wuhu Green Pepper (1),7.58,0
7,342213,Yunnan Lettuces,7.04,0
8,345483,Net Lotus Root (1),8.22,0
9,592508,Luosi Pepper,11.72,0


In [24]:
# -------------------------- predicting future sales --------------------

predicted_future_sales = []

def pfs(date, data):
    weekend = 0
    if pd.to_datetime(date).dayofweek == 5 or pd.to_datetime(date).dayofweek == 6:
        weekend = 1
    # Create a DataFrame instead of a dictionary
    data_df = pd.DataFrame({
        'p_id': [data['p_id']], 
        'p_discount': [data['p_discount']],
        'p_price': [data['p_price']],  # Example Value
        'monthOfyear': [pd.to_datetime(date).month],
        'DayOfWeek': [pd.to_datetime(date).dayofweek],
        'DayOfMonth': [pd.to_datetime(date).day],
        'year': [pd.to_datetime(date).year],
        'Weekend': [weekend],
        'WeekOfMonth': [(pd.to_datetime(date).day - 1) // 7 + 1],
        'quater': [pd.to_datetime(date).quarter]
    })

    predicted_future_sales.append([date, data['p_id'], rf_model.predict(data_df)[0], data['p_discount'], data['p_price']])

from datetime import timedelta, datetime

def sales_forcasting(dt_start, dt_end, discounted):
    while dt_start <= dt_end:
        dt_start += timedelta(days=1)
        for _, row in product.iterrows():
            pfs(dt_start, row)

start_date = datetime.now().date()
end_date = start_date + timedelta(days=30)

sales_forcasting(start_date, end_date, 0)



In [25]:

# Create a DataFrame from the list
predicted_sales_df = pd.DataFrame(predicted_future_sales)
predicted_sales_df.columns = ['Date','p_id','Predicted Sales','Discount','Price']

# Now you have a DataFrame named 'predicted_sales_df' with the specified columns

predicted_sales_df

,Date,p_id,Predicted Sales,Discount,Price
0,2025-01-25,133174,14.92032,0,10.28
1,2025-01-25,151984,7.25477,0,27.18
2,2025-01-25,151990,68.04000,0,3.06
3,2025-01-25,177880,13.16696,0,7.33
4,2025-01-25,184283,42.49152,1,3.41
...,...,...,...,...,...
615,2025-02-24,864181,7.68070,0,18.15
616,2025-02-24,889871,2.43119,0,6.84
617,2025-02-24,916192,7.20315,0,6.87
618,2025-02-24,981080,8.92424,0,8.18


In [28]:
# ----------------------- total stock need for that specific time line ---------------


total_stock_needed = predicted_sales_df.groupby(['p_id'])['Predicted Sales'].sum().reset_index()
total_stock_needed = pd.merge(total_stock_needed, product, on='p_id', how='left')[['p_name', 'Predicted Sales']]
total_stock_needed

,p_name,Predicted Sales
0,Spinach,250.89345
1,Millet Pepper,190.85469
2,Needle Mushroom (Box),1317.45720
3,Hongshujian,442.08440
4,Yunnan Lettuce (Bag),1242.95750
5,Naibaicai,313.39290
6,Wuhu Green Pepper (1),1026.82619
7,Yunnan Lettuces,233.08608
8,Net Lotus Root (1),1121.12653
9,Luosi Pepper,319.14021
